In [19]:
import sys
sys.path.append('../')
from settings import *
import pandas as pd
import networkx as nx

In [25]:
print('MS dataset: ', MSrawdata)
print('Cosine tolerance: ', cosine_tolerance)
print('Similarity cut-off: ', cutoff)
print('Maxmimum edges per node: ', links)

MS dataset:  MassBank_NIST_Feb20
Cosine tolerance:  0.1
Similarity cut-off:  0.6
Maxmimum edges per node:  10


In [21]:
data_path_net = '../data/processed/MolecularNetwork/'

In [22]:
network = nx.read_graphml(data_path_net + MSrawdata+"_tol"+str(cosine_tolerance)+"_cutoff"+str(cutoff)+"_links"+str(links)+".graphml")
tox=pd.read_csv('../data/processed/tox21/tox.csv')
tox.set_index('inchikey', inplace=True)
endpoints=list(tox.columns.values)

In [23]:
from sklearn.metrics import precision_recall_fscore_support
def calc_metrics (endpoint, network):
    
    predictions=pd.DataFrame(None)
    comparison=pd.DataFrame(None)
    predictions = pd.DataFrame(columns=['inchikey', 'predicted_activity'])

    for node in network.nodes:
        neighbors = list(network.neighbors(node))
        filtered_neighbors = [n for n in neighbors if not pd.isna(tox.loc[n].loc[endpoint])]
    
        if len(filtered_neighbors) == 0:
            predicted_activity = float("nan")
        else:
            num_zeros = sum(tox.loc[n].loc[endpoint] == 0 for n in filtered_neighbors)
            num_ones = len(filtered_neighbors) - num_zeros
    
            if num_zeros == num_ones:
                predicted_activity = 1
            else:
                if num_ones > num_zeros:
                    predicted_activity = 1
                else:
                    predicted_activity = 0
    
        temp_df = pd.DataFrame({'inchikey': [node], 'predicted_activity': [predicted_activity]})
        predictions = pd.concat([predictions, temp_df], ignore_index=True)
        
    predictions.set_index('inchikey', inplace=True)
    comparison=predictions.join(tox[endpoint]).dropna().astype('int')

    precision, recall, f1_score, support = precision_recall_fscore_support(comparison[endpoint], comparison['predicted_activity'], pos_label=1, average='binary')
    return endpoint, recall, precision, f1_score

In [24]:
columns = ['endpoint','recall','precision','F1-score']
rows = []
for endpoint in endpoints:
    rows.append(calc_metrics(endpoint, network))
df = pd.DataFrame(rows, columns=columns)
df

,endpoint,recall,precision,F1-score
0,NR.AhR,0.545455,0.526316,0.535714
1,NR.AR,0.818182,0.750000,0.782609
2,NR.AR.LBD,0.666667,0.578947,0.619718
3,NR.Aromatase,0.413793,0.428571,0.421053
4,NR.ER,0.491525,0.397260,0.439394
5,NR.ER.LBD,0.275862,0.296296,0.285714
6,NR.PPAR.gamma,0.500000,0.666667,0.571429
7,SR.ARE,0.408163,0.392157,0.400000
8,SR.ATAD5,0.333333,0.363636,0.347826
9,SR.HSE,0.157895,0.130435,0.142857
